# 1. 연구목표

- MNIST 손글씨 이미지 데이터를 사용
- 손글씨 이미지를 분류
- 알고리즘 목적
  - 딥러닝의 CNN을 이용해서 손글씨를 예측하는 모델 구축
  - 예측 모델 도출, 정확도 평가 후 관찰
- 딥러닝 엔진 중 텐서플로우를 이용해 구현

# 2. 데이터 확보/수집

In [0]:
import tensorflow as tf
%tensorflow_version 1.x

In [0]:
mnist = tf.keras.datasets.mnist.load_data(path='mnist.npz')

In [135]:
type(mnist), len(mnist[0][0]), mnist[0][1], len(mnist[1][0]), mnist[1][1]
# 훈련용 데이터: mnist[0][0]
# 훈련용 레이블: mnist[0][1]
# 테스트용 데이터: mnist[1][0]
# 테스트용 레이블: mnist[1][1]

(tuple,
 60000,
 array([5, 0, 4, ..., 5, 6, 8], dtype=uint8),
 10000,
 array([7, 2, 1, ..., 4, 5, 6], dtype=uint8))

In [136]:
# 레이블이 이미 벡터화 되어서 제공되는 tf의 데이터셋 활용(1.x에서만 사용 가능)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/mnist/', one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [137]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [138]:
mnist.train.images.shape, mnist.train.labels.shape

((55000, 784), (55000, 10))

In [139]:
mnist.train.labels[:3]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [140]:
# 이미지 데이터는 float
mnist.train.images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 3. 데이터 준비

In [141]:
# 벡터화된 레이블을 보고 원래값 0~9를 찾아내는 방법 확인
# 힌트 => 배열 10칸 중에서 1이 존재하는 인덱스 값 => 실제 수치값
type(mnist.train.labels)

numpy.ndarray

In [0]:
import numpy as np

In [143]:
np.where( mnist.train.labels[:1][0] )[0][0]

7

In [144]:
# 이미지 1개당 피쳐의 크기
pixels = mnist.train.images.shape[1]
pixels

784

In [145]:
# 레이블 1개당 피쳐의 크기
nums = mnist.train.labels.shape[1]
nums

10

In [146]:
# 이미지 가로 크기 혹은 세로 크기
pixel_wh_size = int(np.sqrt(pixels))
pixel_wh_size

28

# 4. 데이터 분석(생략)

# 5. 모델링

## 5-1. 데이터 플로우 그래프 구축
- 레이어를 구축하면서 상수, 변수, 플레이스홀더, 연산 등의 요소를 정의
- 레이어 구축 및 연결(원칙: 앞단계의 출력은 뒷단계의 입력이 됨)
  - 입력층
  - 합성곱층a
  - 풀링층a
  - 합성곱층b
  - 풀링층b
  - 전결합층
  - 드롭아웃층
  - 출력층

## 5-2. 실행(학습, 예측, 평가)
- 데이터를 준비(batch) -> 학습 -> 예측 -> 평가(출력)

## 5-1. 데이터 플로우 그래프 구축


### 입력층

In [0]:
# x: 외부에서 손글씨 이미지 데이터가 들어오는 플레이스홀더
# shape(n, 784): 이미지 1개를 표현하는 데 feature 784개 필요
x = tf.placeholder(tf.float32, shape=(None,pixels), name='x')

### 합성곱층a

#### W
- 가중치를 파라미터로 가지는 필터값 필요
- 행렬, 외부에서 주입(X), 내부적인 함수를 활용해 생성 => 텐서의 타입 중 Varibale
- 2개의 합성곱층에서 W가 각각 필요 => W를 만드는 함수 필요

In [0]:
# shape: 가중치를 공용 파라미터로 가지는 필터(=커널)의 shape / 커널의 크기가 무조건 3x3은 아님
# name:  각 구성원들의 이름을 구분하기 위해서 직접 부여
def makeWeightVariable(shape, name):
  # 변수의 초기값 -> 절단 정규분포를 통한 난수를 발생하는 함수 활용
  # shape 크기에 맞춰서 배열을 만들고, 함수가 계산해서 난수를 배치함(커널 크기만큼 행렬이 만들어지고, 난수가 설정됨) 
  W_initValue = tf.truncated_normal(shape, stddev=0.1)
  W = tf.Variable(W_initValue, name=f'W_{name}')
  return W

In [0]:
# bias 초기화 함수
# 변수로 정의하고 고정 임계값으로 0.1을 임시 부여
def makeBiasVariable(shape, name):
  b_initValue = tf.constant(0.1, shape=[shape])
  b = tf.Variable(b_initValue, name=f'b_{name}')
  return b

In [150]:
# 합성곱 계층을 만드는 함수
def makeConv2d(x, W):
  conv2d = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
  return conv2d

'''
tf.nn.conv2d(
    input: x => [batch: 1회에 훈련하고자 하는 이미지의 총 개수, 
                in_height: 세로 크기, in_width: 가로 크기, 
                in_channels: color에 따라 다름]
    filters:(=kernel, 필터의 공용 파라미터, 필터를 구성하는 구성원의 값 W)
            => [filter_height, filter_width: 필터의 크기, 
                in_channels: 입력채널수 -> 1, 
                out_channels: 출력채널수 -> 임의의 값]
    strides: {int scalar}, {int list} lenght=1 or 2 or 4
            => [batch, w, h, depth] batch, depth는 통상적으로 1 사용 / w, h는 일반적으로 같은 값
            => [1, 1, 1, 1]
    padding: 특성맵 보정, 외곽에서 개수가 부족할수도 있는데 이를 통상 0으로 테두리 보정(SAME)
    data_format='NHWC',
    dilations=None,
    name=None
)
'''

"\ntf.nn.conv2d(\n    input: x => [batch: 1회에 훈련하고자 하는 이미지의 총 개수, \n                in_height: 세로 크기, in_width: 가로 크기, \n                in_channels: color에 따라 다름]\n    filters:(=kernel, 필터의 공용 파라미터, 필터를 구성하는 구성원의 값 W)\n            => [filter_height, filter_width: 필터의 크기, \n                in_channels: 입력채널수 -> 1, \n                out_channels: 출력채널수 -> 임의의 값]\n    strides: {int scalar}, {int list} lenght=1 or 2 or 4\n            => [batch, w, h, depth] batch, depth는 통상적으로 1 사용 / w, h는 일반적으로 같은 값\n            => [1, 1, 1, 1]\n    padding: 특성맵 보정, 외곽에서 개수가 부족할수도 있는데 이를 통상 0으로 테두리 보정(SAME)\n    data_format='NHWC',\n    dilations=None,\n    name=None\n)\n"

In [0]:
# 합성곱층a 생성: 텐서보드 상에 작업 공간을 잡음
with tf.name_scope('conv1') as scope:
  # W / shape=[filter_height, filter_width, in_channels, out_channels]
  W_conv1 = makeWeightVariable([5, 5, 1, 32], name='conv1')
  # b / b의 shape = W의 out_channels
  b_conv1 = makeBiasVariable(32, name='conv1')
  # x => [batch, h, w, channels]
  x_image = tf.reshape(x, [-1, pixel_wh_size, pixel_wh_size, 1])

  # 컨볼루션 레이어 생성
  # 활성화 함수를 통과시켜서 레이어의 출력물(특성맵)을 비선형으로 보정(활성화맵)
  h_conv1 = tf.nn.relu(makeConv2d(x_image, W_conv1) + b_conv1)

In [152]:
h_conv1.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(32)])

### 풀링층a

- 특성맵(or 활성화맵)의 특성을 강화
- 최대풀링, 평균풀링, 최소풀링 등 여러개 값 중 하나를 취해서 행렬을 새로 구성(축소)
- 샘플링한다고 표현
- 크기에 관여 => 얼마만큼 이동하면서 처리할 것인가(stride)
- 커널에 파라미터는 없지만 최대/평균/최소 등의 의미는 가짐
- input: 합성곱층a의 결과(h_conv1)

In [153]:
def makeMaxPooling(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

'''
  tf.nn.max_pool(
    value: 이전 계층의 출력값=>[batch, h, w, channels],
    ksize: 풀링 수행을 위한 슬라이딩 윈도우(=커널,필터)의 크기=>[, h, w, ],
    strides: ksize의 이동 폭=>[batch, w, h, depth],
    # 일반적으로 ksize와 strides 동일하게 준다
    padding: 보정,
  )
'''

'\n  tf.nn.max_pool(\n    value: 이전 계층의 출력값=>[batch, h, w, channels],\n    ksize: 풀링 수행을 위한 슬라이딩 윈도우(=커널,필터)의 크기=>[, h, w, ],\n    strides: ksize의 이동 폭=>[batch, w, h, depth],\n    # 일반적으로 ksize와 strides 동일하게 준다\n    padding: 보정,\n  )\n'

In [0]:
# 풀링층a 생성
with tf.name_scope('pool1') as scope:
  h_pool1 = makeMaxPooling(h_conv1)

In [155]:
h_pool1.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(32)])

### 합성곱층b

In [0]:
# 레이어를 하나 더 쌓음
with tf.name_scope('conv2') as scope:
  W_conv2 = makeWeightVariable([5, 5, 32, 64], name='conv2')
  b_conv2 = makeBiasVariable(64, name='conv2')
  h_conv2 = tf.nn.relu(makeConv2d(h_pool1, W_conv2) + b_conv2)

In [157]:
h_conv2.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(64)])

### 풀링층b

In [0]:
with tf.name_scope('pool2') as scope:
  h_pool2 = makeMaxPooling(h_conv2)

In [159]:
h_pool2.shape

TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(64)])

### 전결합층

- 원본 이미지는 풀링계층을 두번 통과하면서 크기가 28/2/2 = 7이 됨
- 출력층으로 가기 직전 단계로써, 행렬 데이터를 Flatten(h, w, c를 한 줄의 데이터로 펴줌)하는 것이 목적
- 최종 데이터: [n, size(조정가능)]

In [0]:
with tf.name_scope('fully_connected') as scope:
  # 이미지 데이터 size => n
  n = 7 * 7 * 64
  # 1024는 임의의 설정 값: n(=3136) -> 1024
  W_fc = makeWeightVariable([n,1024], name='fc')
  b_fc = makeBiasVariable(1024, name='fc')
  # 이전 계층의 출력 데이터를 flatten 처리
  h_pool2_flat = tf.reshape(h_pool2, [-1, n])
  h_fc = tf.nn.relu( tf.matmul( h_pool2_flat, W_fc ) + b_fc )


In [161]:
h_fc.shape

TensorShape([Dimension(None), Dimension(1024)])

### 드롭아웃층

- 과적합 방지
- 랜덤하게 뉴런을 일부 죽여서 학습을 방해


In [0]:
with tf.name_scope('dropout1') as scope:
  keep_prob = tf.placeholder(tf.float32)
  h_fc_drop = tf.nn.dropout(h_fc, rate = 1 - keep_prob)

In [163]:
h_fc_drop

<tf.Tensor 'dropout1/dropout/mul_1:0' shape=(?, 1024) dtype=float32>

### 출력층

- 이전 단계의 출력 데이터를 최종 레이블의 shape로 맞춘다(1024->10)
- 이 과정에서 활성화 함수 사용
  - softmax 사용
    - 입력을 0~1로 정규화 해 출력
    - 모든 값들의 총합은 1 -> 특정 값이 나오는 확률(%)로 표현 가능
    - 일반적으로 뉴럴네트워크에서는 sigmoid를 사용
    - 3가지 이상 범주형 분류(다항분류)의 경우 softmax를 가장 많이 사용


In [0]:
with tf.name_scope('output') as scope:
  W_out = makeWeightVariable([1024,10], name='output')
  b_out = makeBiasVariable(10, name='output')
  y_conv = tf.nn.softmax( tf.matmul( h_fc_drop, W_out ) + b_out )

In [165]:
y_conv

<tf.Tensor 'output/Softmax:0' shape=(?, 10) dtype=float32>

In [0]:
# 이후 실행에 필요한 정답 벡터가 들어갈 플레이스홀더 준비
y_ = tf.placeholder(tf.float32, shape=(None, nums), name='y_')

## 5-2. 실행(학습, 예측, 평가)

### 크로스 엔트로피, 손실

- 비용(cost), 손실(loss) 등은 원하는 결과에서 얼마나 떨어져 있는지 보여주는 지표
- 이 격차를 줄이는 방향으로 전개

In [0]:
with tf.name_scope('loss') as scope:
  cross_entropy = -tf.reduce_sum( y_ * tf.log(y_conv) )

### 경사 하강법

- 크로스엔트로피의 값(손실함수의 값)이 작아지도록 이동시키는 방법
- 확률적 경사하강법
  - 무작위로 초기화한 매개변수를 이용

In [0]:
with tf.name_scope('sgd') as scope:
  optimizer = tf.train.AdamOptimizer(1e-4)
  train_step = optimizer.minimize(cross_entropy)

### 예측, 평가

In [0]:
with tf.name_scope('predict') as scope:
  predict_step = tf.equal( tf.arg_max(y_conv, 1), tf.arg_max(y_, 1) )
  accuracy_step = tf.reduce_mean( tf.cast(predict_step, tf.float32) )

### 데이터 주입 함수화

- 데이터 플로우 그래프 전체에서 placeholder를 모두 찾아서 명시
- x, y, keep_prob


In [0]:
def makeFeedDict( imgs, labels, prob ):
  return { x:imgs, y_:labels, keep_prob:prob}

### 실행

In [171]:
from tensorboardcolab import *
import shutil, os
# 디렉토리 및 그 이하 파일가지 삭제, 에러나면 무시
shutil.rmtree('./Graph', ignore_errors=True) 
# 디렉토리 생성
os.mkdir('./Graph')
# 기존의 tf에 만들어진 요소들 초기화 처리(에러가 날수도 있다)
# tf.reset_default_graph()
# 텐서보드 객체 생성
tbc = TensorBoardColab()

TRAIN_CNT = 3000 #학습총횟수
with tf.Session() as sess:
  # 1. 텐서플로우 변수 초기화
  sess.run(tf.global_variables_initializer())
  # 2. 테스트용 데이터 준비
  test_fd = makeFeedDict(mnist.test.images, mnist.test.labels, 1)
  # 3. 반복학습
  for step in range(TRAIN_CNT):
    # 3-1. batch 사이즈(1회 기준.gpu 메모리 계산) 기준으로 훈련 데이터를 준비
    batch = mnist.train.next_batch(batch_size=50)
    # 3-2. 훈련용 데이터 준비
    train_fd = makeFeedDict(batch[0], batch[1], 0.5)
    # 3-3. 훈련(데이터 주입)
    _, loss = sess.run([train_step, cross_entropy], feed_dict=train_fd)
    # 3-4. 특정 횟수마다 로그 출력
    if step % 100 == 0:
      accuracy = sess.run(accuracy_step, feed_dict=test_fd)
      print(f'{step}번째 정확도:{accuracy}/손실:{loss}')
  # 4. 출력
  f_acc = sess.run(accuracy_step, feed_dict=test_fd)
  print(f'최종 정답률:{accuracy}')
  # 5. 텐서보드 기록
  writer = tbc.get_writer()
  writer.add_graph( sess.graph ) # 그래프 추가=
  writer.flush() # 강제로 전송
  # ---------------------------------------------------------------------

# 텐서보드를 닫는다 
tbc.close()

Wait for 8 seconds...
TensorBoard link:
https://4a4a786a.ngrok.io
0번째 정확도:0.16619999706745148/손실:496.42242431640625
100번째 정확도:0.8421000242233276/손실:45.937103271484375
200번째 정확도:0.9151999950408936/손실:21.51087188720703
300번째 정확도:0.9259999990463257/손실:33.51099395751953
400번째 정확도:0.946399986743927/손실:12.881881713867188
500번째 정확도:0.9520000219345093/손실:10.291302680969238
600번째 정확도:0.957099974155426/손실:15.444109916687012
700번째 정확도:0.9607999920845032/손실:9.907418251037598
800번째 정확도:0.9628999829292297/손실:7.703657627105713
900번째 정확도:0.9678999781608582/손실:4.632031440734863
1000번째 정확도:0.967199981212616/손실:11.409454345703125
1100번째 정확도:0.9703999757766724/손실:9.504620552062988
1200번째 정확도:0.9710999727249146/손실:11.781147956848145
1300번째 정확도:0.9726999998092651/손실:3.939415693283081
1400번째 정확도:0.9724000096321106/손실:3.0913968086242676
1500번째 정확도:0.9763000011444092/손실:9.12407112121582
1600번째 정확도:0.9764000177383423/손실:6.6935014724731445
1700번째 정확도:0.9750999808311462/손실:2.344017505645752
1800번째 정확도:0.977500021

# 6. 시스템 통합(생략)